In [ ]:
import omero
from OMERO_code import omero_tools
import os
import sys
import json
from collections import OrderedDict

# connect to OMERO
conn = omero_tools.get_connection()
conn.SERVICE_OPTS.setOmeroGroup(omero_tools.project_gid)

# helper function to parse OMERO datasets
def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print """%s%s:%s  Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName())

# get datasetname (need to make datasetid an argument)
dataset_id = 17759
dataset = conn.getObject("Dataset", dataset_id)
dataset_name = dataset.getName()

# store image names and their IDs in a dict
image_name_ID = {}
for image in conn.getObject("Dataset", dataset_id).listChildren():
        image_name_ID.update({image.getname() : image.getId()})

# Need to supply dict with gene names and CPTI numbers
CPTI_genes = OrderedDict()
CPTI_genes['CPTI001308'] = 'CG32479'
CPTI_genes['CPTI001586'] = 'CG31738'
CPTI_genes['CPTI001771'] = 'CG31694'
CPTI_genes['CPTI003100'] = 'CG32062'

def make_figure(data):
    # replace statements that are common to all figures
    data = data.replace('20180710_CPTI_AMPdeaminase', dataset_name) # dataset name
    data = data.replace('11402', str(dataset_id)) # dataset ID
    data = data.replace('AMP-deaminase', gene) # figure name
    data = data.replace('CPTI001081', CPTI) # figure name

    # convert Python string to json dict
    figure_json = json.loads(data)

    # create a json text file for OMERO file_ann
    with open('json_data.txt', 'w') as fh:
        fh.write(data)
    fh.close()
    figure_fpath = 'json_data.txt'

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# ------------------------------------------------------------------------------
#   Copyright (C) 2017-2018 University of Dundee. All rights reserved.

#   This program is free software; you can redistribute it and/or modify
#   it under the terms of the GNU General Public License as published by
#   the Free Software Foundation; either version 2 of the License, or
#   (at your option) any later version.
#   This program is distributed in the hope that it will be useful,
#   but WITHOUT ANY WARRANTY; without even the implied warranty of
#   MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
#   GNU General Public License for more details.

#   You should have received a copy of the GNU General Public License along
#   with this program; if not, write to the Free Software Foundation, Inc.,
#   51 Franklin Street, Fifth Floor, Boston, MA 02110-1301 USA.

# ------------------------------------------------------------------------------

"""
Simple FRAP plots from Ellipses on images and creates an OMERO.figure.
This an OMERO script that runs server-side.
"""

import omero
import json
from cStringIO import StringIO

import omero.scripts as scripts
from omero.rtypes import rlong, rstring, unwrap
from omero.gateway import BlitzGateway

from PIL import Image
import numpy as np
try:
    import matplotlib
    # Run headless, so that pyplot doesn't try to import gtk
    matplotlib.use('Agg')
    from matplotlib import pyplot as plt
except (ImportError, RuntimeError):
    plt = None

JSON_FILEANN_NS = "omero.web.figure.json"


def channelMarshal(channel):
    """
    Return a dict with all there is to know about a channel.
    NB: This is copied from omeroweb.webgateway.marshal.py since we don't know
    that OMERO.web is installed on same environment as scripts
    @param channel:     L{omero.gateway.ChannelWrapper}
    @return:            Dict
    """
    chan = {'emissionWave': channel.getEmissionWave(),
            'label': channel.getLabel(),
            'color': channel.getColor().getHtml(),
            # 'reverseIntensity' is deprecated. Use 'inverted'
            'inverted': channel.isInverted(),
            'reverseIntensity': channel.isInverted(),
            'family': unwrap(channel.getFamily()),
            'coefficient': unwrap(channel.getCoefficient()),
            'window': {'min': channel.getWindowMin(),
                       'max': channel.getWindowMax(),
                       'start': channel.getWindowStart(),
                       'end': channel.getWindowEnd()},
            'active': channel.isActive()}
    lut = channel.getLut()
    if lut and len(lut) > 0:
        chan['lut'] = lut
    return chan


def get_timestamps(conn, image):
    """Return a list of times (secs) 1 for each T-index in image."""
    params = omero.sys.ParametersI()
    params.addLong('pid', image.getPixelsId())
    query = "from PlaneInfo as Info where"\
        " Info.theZ=0 and Info.theC=0 and pixels.id=:pid"
    info_list = conn.getQueryService().findAllByQuery(
        query, params, conn.SERVICE_OPTS)
    timemap = {}
    for info in info_list:
        t_index = info.theT.getValue()
        if info.deltaT is not None:
            delta_t = info.deltaT.getValue()
            timemap[t_index] = round(delta_t, 2)
    time_list = []
    for t in range(image.getSizeT()):
        if t in timemap:
            time_list.append(timemap[t])
    return time_list


def create_figure_file(conn, figure_json):
    """Create Figure FileAnnotation from json data."""
    figure_name = figure_json['figureName']
    if len(figure_json['panels']) == 0:
        raise Exception('No Panels')
    first_img_id = figure_json['panels'][0]['imageId']

    # we store json in description field...
    description = {}
    description['name'] = figure_name
    description['imageId'] = first_img_id

    # Try to set Group context to the same as first image
    conn.SERVICE_OPTS.setOmeroGroup('-1')
    i = conn.getObject("Image", first_img_id)
    gid = i.getDetails().getGroup().getId()
    conn.SERVICE_OPTS.setOmeroGroup(gid)

    json_string = json.dumps(figure_json)
    file_size = len(json_string)
    f = StringIO()
    json.dump(figure_json, f)

    update = conn.getUpdateService()
    orig_file = conn.createOriginalFileFromFileObj(
        f, '', figure_name, file_size, mimetype="application/json")
    fa = omero.model.FileAnnotationI()
    fa.setFile(omero.model.OriginalFileI(orig_file.getId(), False))
    fa.setNs(rstring(JSON_FILEANN_NS))
    desc = json.dumps(description)
    fa.setDescription(rstring(desc))
    fa = update.saveAndReturnObject(fa, conn.SERVICE_OPTS)
    return fa.getId().getValue()


def get_scalebar_json():
    """Return JSON to add a 10 micron scalebar to bottom-right."""
    return {"show": True,
            "length": 10,
            "position": "bottomright",
            "color": "FFFFFF",
            "show_label": True,
            "font_size": "10"}


def get_panel_json(image, x, y, width, height, theT):
    """Get json for a figure panel."""
    px = image.getPrimaryPixels().getPhysicalSizeX()
    py = image.getPrimaryPixels().getPhysicalSizeY()

    channels = map(lambda x: channelMarshal(x), image.getChannels())

    img_json = {
        "labels": [],
        "height": height,
        "channels": channels,
        "width": width,
        "sizeT": image.getSizeT(),
        "sizeZ": image.getSizeZ(),
        "dx": 0,
        "dy": 0,
        "rotation": 0,
        "imageId": image.id,
        "name": image.getName(),
        "orig_width": image.getSizeX(),
        "zoom": 100,
        "shapes": [],
        "orig_height": image.getSizeY(),
        "y": y,
        "x": x,
        "theT": theT,
        "theZ": 0
    }
    if px is not None:
        img_json["pixel_size_x"] = px.getValue()
        img_json["pixel_size_x_unit"] = str(px.getUnit())
        img_json["pixel_size_x_symbol"] = px.getSymbol()
    if py is not None:
        img_json["pixel_size_y"] = py.getValue()
    if image.getSizeT() > 1:
        img_json['deltaT'] = get_timestamps(conn, image)
    return img_json


def create_omero_figure(conn, images, plots):
    """Create OMERO.figure from given FRAP images and plot images."""
    figure_json = {"version": 2,
                   "paper_width": 595,
                   "paper_height": 842,
                   "page_size": "A4",
                   "figureName": "FRAP figure from script",
                   }
    time_frames = [2, 3, 4, 5, 10]

    panel_width = 80
    panel_height = panel_width
    spacing = panel_width/20
    margin = 40

    panels_json = []

    for i, image in enumerate(images):

        panel_x = margin
        panel_y = (i * (panel_height + spacing)) + margin
        for col in range(len(time_frames)):
            the_t = time_frames[col]
            panel_x = (col * (panel_height + spacing)) + margin
            j = get_panel_json(image, panel_x, panel_y,
                               panel_width, panel_height, the_t)
            # Add timestamp in 'secs' to top-left of each movie frame
            j['labels'] = [{"time": "secs",
                            "size": "12",
                            "position": "topleft",
                            "color": "FFFFFF"}]
            panels_json.append(j)
        # Add scalebar to last panel
        panels_json[-1]['scalebar'] = get_scalebar_json()
        # Add plot
        if i < len(plots):
            plot = plots[i]
            panel_x = (len(time_frames) * (panel_height + spacing)) + margin
            plot_width = (panel_height *
                          (float(plot.getSizeX()) / plot.getSizeY()))
            j = get_panel_json(plot, panel_x, panel_y,
                               plot_width, panel_height, 0)
            panels_json.append(j)

    figure_json['panels'] = panels_json
    return create_figure_file(conn, figure_json)


def run(conn, params):
    """
    For each image, getTiles() for FRAP Ellipse and plot mean intensity.
    Returns list of images
    @param conn   The BlitzGateway connection
    @param params The script parameters
    """
    images = []

    if params.get("Data_Type") == 'Dataset':
        for dsId in params["IDs"]:
            dataset = conn.getObject("Dataset", dsId)
            if dataset:
                for image in dataset.listChildren():
                    images.append(image)
    elif params.get("Data_Type") == 'Image':
        images = list(conn.getObjects('Image', params["IDs"]))

    if len(images) == 0:
        return None
    roi_service = conn.getRoiService()

    frap_plots = []

    for image in images:
        print "---- Processing image", image.id
        result = roi_service.findByImage(image.getId(), None)

        # Simply use any Ellipse we find...
        shape_id = None
        for roi in result.rois:
            print "ROI:  ID:", roi.getId().getValue()
            for s in roi.copyShapes():
                if type(s) == omero.model.EllipseI:
                    shape_id = s.id.val
        print "Shape:", shape_id
        if shape_id is None:
            print "  No Ellipse found for this image"
            continue

        # Get pixel intensities for first Channel
        the_c = 0
        the_z = 0
        size_t = image.getSizeT()
        meanvalues = []
        for t in range(size_t):
            stats = roi_service.getShapeStatsRestricted([shape_id],
                                                        the_z, t, [the_c])
            meanvalues.append(stats[0].mean[the_c])

        print meanvalues

        # Add values as a Map Annotation on the image
        key_value_data = [[str(t), str(meanvalues[t])] for t in range(size_t)]
        map_ann = omero.gateway.MapAnnotationWrapper(conn)
        namespace = "demo.simple_frap_data"
        map_ann.setNs(namespace)
        map_ann.setValue(key_value_data)
        map_ann.save()
        image.linkAnnotation(map_ann)

        if plt is not None:
            # Code from https://stackoverflow.com/questions/7821518/
            fig = plt.figure()
            plt.subplot(111)
            plt.plot(meanvalues)
            fig.canvas.draw()
            fig.savefig('plot.png')
            pil_img = Image.open('plot.png')
            np_array = np.asarray(pil_img)
            red = np_array[::, ::, 0]
            green = np_array[::, ::, 1]
            blue = np_array[::, ::, 2]
            plane_gen = iter([red, green, blue])
            plot_name = image.getName() + "_FRAP_plot"
            i = conn.createImageFromNumpySeq(plane_gen, plot_name, sizeC=3,
                                             dataset=image.getParent())
            frap_plots.append(i)

    return create_omero_figure(conn, images, frap_plots)


if __name__ == "__main__":
    dataTypes = [rstring('Dataset'), rstring('Image')]
    client = scripts.client(
        'Simple_FRAP_with_figure.py',
        """
    This script does simple FRAP analysis using Ellipse ROIs previously
    saved on images. If matplotlib is installed, data is plotted and new
    OMERO images are created from the plots.
        """,
        scripts.String(
            "Data_Type", optional=False, grouping="1",
            description="Choose source of images",
            values=dataTypes, default="Dataset"),

        scripts.List(
            "IDs", optional=False, grouping="2",
            description="Dataset or Image IDs.").ofType(rlong(0))
    )

    try:
        # process the list of args above.
        scriptParams = {}
        for key in client.getInputKeys():
            if client.getInput(key):
                scriptParams[key] = client.getInput(key, unwrap=True)
        print scriptParams

        # wrap client to use the Blitz Gateway
        conn = BlitzGateway(client_obj=client)
        # Call the main script - returns the new OMERO.figure ann ID
        figure_id = run(conn, scriptParams)
        if figure_id is None:
            message = "No images found"
        else:
            message = "Created FRAP figure: %s" % figure_id

        client.setOutput("Message", rstring(message))

    finally:
        client.closeSession()

In [249]:
import omero
from omero.gateway import BlitzGateway
import os
import getpass
import threading
import time
import os
from collections import OrderedDict
import json


# connect to OMERO
USER = os.environ['USER']
PASS = getpass.getpass("Enter Password:")
HOST = os.environ['HOST']

HOST = "omero1.bioch.ox.ac.uk"
conn = BlitzGateway(USER, PASS, host=HOST, port=4064)
conn.connect()
conn.SERVICE_OPTS.setOmeroGroup(-1)

# keep connection to OMERO alive
def keep_connection_alive():
    while True:
        conn.keepAlive()
        time.sleep(60)

th_ka = threading.Thread(target = keep_connection_alive)
th_ka.daemon = True
th_ka.start()

# helper function to parse OMERO datasets
def print_obj(obj, indent=0):
    """
    Helper method to display info about OMERO objects.
    Not all objects will have a "name" or owner field.
    """
    print """%s%s:%s  Name:"%s" (owner=%s)""" % (
        " " * indent,
        obj.OMERO_CLASS,
        obj.getId(),
        obj.getName(),
        obj.getOwnerOmeName())

# get datasetname (need to make datasetid an argument)
dataset_id = 17759
dataset = conn.getObject("Dataset", dataset_id)
dataset_name = dataset.getName()

# store image names and their IDs as a dictionary
image_name_ID = {}
for image in conn.getObject("Dataset", dataset_id).listChildren():
        image_name_ID.update({image.getname() : image.getId()})


# templates (hard-coded)
#MBtemplate = open('/Users/joshtitlow/Desktop/DavidGUI/json_Figure/template_json_mb.txt','r')
#overviewtemplate= open('/Users/joshtitlow/Desktop/DavidGUI/json_Figure/template_json_overview.txt')

# Arguments
CPTI_genes = OrderedDict()
CPTI_genes['CPTI001308'] = 'geneA'
CPTI_genes['CPTI001586'] = 'geneB'
CPTI_genes['CPTI001771'] = 'geneC'
CPTI_genes['CPTI003100'] = 'geneD'

# items to replace
filename= 'AMP-deaminase_CPTI001081_VNC_venusYFP_zegami1'

for CPTI, gene in CPTI_genes.items():
    
    # MB figure
    # load template json text and replace filenames
    MBtemplate = open('template_json_mb.txt','r')
    dataimage_dorsal = [dd for dd in image_name_ID if CPTI in dd and 'dorsal' in dd]
    dataimage_ventral = [dv for dv in image_name_ID if CPTI in dv and 'ventral' in dv]
    controlimage_dorsal = [cd for cd in image_name_ID if 'OrR' in cd and 'dorsal' in cd]
    controlimage_ventral = [cv for cv in image_name_ID if 'OrR' in cv and 'ventral' in cv]
    
    lines = MBtemplate.readlines()
    #print(len(lines))
    m = lines[0]
    m1 = m.replace('20180710_CPTI_AMPdeaminase', dataset_name) # dataset name
    m2 = m1.replace('11402', dataset_id) # dataset ID
    m3 = m2.replace('CPTI_AMPdeaminase_lobe_dorsal.oif', dataimage_dorsal) # dorsal data
    m4 = m3.replace('OrR_CPTI_AMPdeaminase_lobe_dorsal.oif', controlimage_dorsal) # dorsal control
    m5 = m4.replace('CPTI_AMPdeaminase_lobe_ventral.oif', dataimage_ventral) # ventral data
    m6 = m5.replace('OrR_CPTI_AMPdeaminase_lobe_ventral.oif', controlimage_ventral) # ventral control
    m7 = m6.replace('AMP-deaminase', gene) # figure name
    m8 = m7.replace('CPTI001081', CPTI) # figure name
    
    m9 = s8.replace('525318', CPTI) # ventral control image ID
   
    # overview figure
    dataimage_o = [do for do in image_name_ID if CPTI in do and 'overview' in do]
    dataID_o = CPTI_genes.get(dataimage_o)
    controlimage = [co for co in image_name_ID if 'OrR' in co and 'dorsal' in co]
    overviewtemplate= open('/Users/joshtitlow/Desktop/DavidGUI/json_Figure/template_json_overview.txt')
    o = overviewtemplate.readlines()
    o1 = o.replace('20180710_CPTI_AMPdeaminase', dataset_name) # dataset name
    o2 = o1.replace('11402', dataset_id) # dataset ID
    o3 = o2.replace('20180709_CPTI_AMPdeaminase_YFP_syp_20x_dorsal_overview.oir', dataimage_o) # overiew data
    o4 = o3.replace('525317', dataID_o) # overview ID
    o5 = o4.replace('AMP-deaminase', gene) # figure name
    #with open('data.json', 'w') as outfile:
        #json.dump(s7, outfile)
    #print outfile
    print o5
conn.close()

# TO DO 
# make the sequential replace statements more elegant
# replace dataset ID and image ID
# delete 'file id' from the template files
# generate new OMERO.Figure from json
#    > this could work by using the import from json command

{"version":3,"panels":[{"z_end":17,"labels":[{"color":"FFFFFF","position":"topleft","text":"RNA","size":12}],"height":180.25174051875356,"channels":[{"coefficient":1,"family":"linear","color":"FFFFFF","emissionWave":null,"inverted":false,"window":{"max":4095,"end":3000,"start":100,"min":12},"active":false,"label":"HSD1","reverseIntensity":false},{"coefficient":1,"family":"linear","color":"FFFFFF","emissionWave":null,"inverted":false,"window":{"max":2094,"end":700,"start":100,"min":13},"active":true,"label":"HSD2","reverseIntensity":false},{"coefficient":1,"family":"linear","color":"00FF00","emissionWave":null,"inverted":false,"window":{"max":2255,"end":1500,"start":200,"min":136},"active":false,"label":"HSD1","reverseIntensity":false},{"coefficient":1,"family":"linear","color":"0000FF","emissionWave":null,"inverted":false,"window":{"max":4095,"end":2000,"start":100,"min":98},"active":false,"label":"HSD2","reverseIntensity":false}],"deltaT":[],"z_start":13,"selected":true,"width":181.26

In [ ]:
import json
import os.path
import sys

import omero_tools
from Figure_To_Pdf import TiffExport


def main(dir_path, metadata_fpath):
    conn = omero_tools.get_connection()
    conn.SERVICE_OPTS.setOmeroGroup(-1)

    metadata = [line.split(',') for line in open(metadata_fpath, 'r')]
    for fig_metadata in metadata:
        fig_id = int(fig_metadata[0])

        json_path = os.path.join(dir_path, '%d.json' % fig_id)
        with open(json_path, 'r') as fh:
            fig_text = fh.read()

        fig_json = json.loads(fig_text)
        if int(fig_json['page_count']) > 1:
            raise RuntimeError("more than one page for figure id '%d'" % fig_id)

        export_params = {
            'Figure_JSON' : fig_text,
            'Webclient_URI': 'https://omero1.bioch.ox.ac.uk',
            'Export_Option' : 'TIFF', # change to jpeg
        }
        fig_export = TiffExport(conn, export_params,
                                export_images=False)
        def get_figure_file_name(page=None):
            return os.path.join(dir_path, '%d.jpg' % fig_id)
        fig_export.get_figure_file_name = get_figure_file_name
        fig_export.build_figure()


if __name__ == '__main__':
    main(*sys.argv[1:])

In [ ]:
lines = /Users/joshtitlow/Desktop/DavidGUI/json_Figure/template_json.txt.read().split(',')
f = open(path,'r')
while True:
    text = f.readline()
    if 'rawr' in text:
        print text

In [7]:
# templates (hard-coded)
MBtemplate = open('/Users/joshtitlow/Desktop/DavidGUI/json_Figure/template_json_mb.txt','r')

# Arguments
#datasetname = '20180810_smFISH_screen_Cycle'
CPTIlines = 'CPTI001308','CPTI001586','CPTI001771','CPTI003100'
dataset ='/Users/joshtitlow/Desktop/DavidGUI/json_Figure/'
image_names = os.listdir(dataset)
#image_names = [i for i in os.listdir(image_dir) if i.endswith('.tif')]

for i in CPTIlines:
    MB_filename = []
    # make MB figure
    lines = MBtemplate.readlines()
    #print(len(lines))
    
    
    s = lines[0]
    s1 = s.replace('20180709_CPTI2645_vha55', 'newdataset')

In [177]:
filename= 'AMP-deaminase_CPTI001081_VNC_venusYFP_zegami1'


CPTI_genes = OrderedDict( 
    'CPTI001308' : 'geneA',
    'CPTI001586' : 'geneB',
    'CPTI001771' : 'geneC',
    'CPTI003100' : 'geneD'
    )
for i in CPTI_genes.keys()
    s=filename.replace('AMP-deaminase_CPTI001081', str(CPTI_genes.values))
    print s
CPTI = CPTI_genes.keys()
genes = CPTI_genes.values()
print genes
    

SyntaxError: invalid syntax (<ipython-input-177-de9712707224>, line 10)

In [239]:
# This works
from collections import OrderedDict

CPTI_genes = OrderedDict()
CPTI_genes['CPTI001308'] = 'geneA'
CPTI_genes['CPTI001586'] = 'geneB'
CPTI_genes['CPTI001771'] = 'geneC'
CPTI_genes['CPTI003100'] = 'geneD'

filename= 'AMP-deaminase_CPTI001081_VNC_venusYFP_zegami1'
print filename

# 
for CPTI, gene in CPTI_genes.items():
    s=filename.replace('AMP-deaminase', gene)
    s1=s.replace('CPTI001081', CPTI)
    print s1

AMP-deaminase_CPTI001081_VNC_venusYFP_zegami1
geneA_CPTI001308_VNC_venusYFP_zegami1
geneB_CPTI001586_VNC_venusYFP_zegami1
geneC_CPTI001771_VNC_venusYFP_zegami1
geneD_CPTI003100_VNC_venusYFP_zegami1


In [214]:
from collections import OrderedDict

CPTI_genes = OrderedDict()
CPTI_genes['CPTI001308'] = 'geneA'
CPTI_genes['CPTI001586'] = 'geneB'
CPTI_genes['CPTI001771'] = 'geneC'
CPTI_genes['CPTI003100'] = 'geneD'

filename= 'AMP-deaminase_CPTI001081_VNC_venusYFP_zegami1'
print filename
for i in CPTI_genes.values():
    s=filename.replace('AMP-deaminase_CPTI001081', str(i))
    print s


AMP-deaminase_CPTI001081_VNC_venusYFP_zegami1
geneA_VNC_venusYFP_zegami1
geneB_VNC_venusYFP_zegami1
geneC_VNC_venusYFP_zegami1
geneD_VNC_venusYFP_zegami1


In [207]:
import os

dataset ='/Users/joshtitlow/Desktop/DavidGUI/json_Figure/'
images = os.listdir(dataset)

matches = [x for x in images if 'CPTI001308' in x and 'dorsal' in x]
matches

['20180810_CPTI001308_dorsal.oir']

In [118]:
# templates (hard-coded)
MBtemplate = open('/Users/joshtitlow/Desktop/DavidGUI/json_Figure/template_json_mb.txt','r')


#MB_filename = []
# make MB figure
lines = MBtemplate.readlines()
print(len(lines))


s = lines[0]
#s1 = s.replace('20180709_CPTI2645_vha55', 'newdataset')

1


In [113]:
print(len(lines))

0


In [45]:
import os

dataset ='/Users/joshtitlow/Desktop/DavidGUI/json_Figure/'

images = os.listdir(dataset)
images

['20180810_CPTI001308_dorsal.oir',
 '20180810_CPTI001308_overview.oir',
 '20180810_CPTI001308_ventral.oir',
 '20180810_CPTI001308_VNC.oir',
 'template_json.txt',
 'template_json_cb.txt',
 'template_json_mb.txt',
 'template_json_ol.txt',
 'template_json_overview.txt',
 'template_json_VNC.txt']

In [36]:
image_names

[False]

In [42]:
MBtemplate = open('/Users/joshtitlow/Desktop/DavidGUI/json_Figure/template_json_mb.txt','r')
lines = MBtemplate.readlines()
lines

['{"version":3,"panels":[{"z_end":17,"labels":[{"color":"FFFFFF","position":"topleft","text":"RNA","size":12}],"height":180.25174051875356,"channels":[{"coefficient":1,"family":"linear","color":"FFFFFF","emissionWave":null,"inverted":false,"window":{"max":4095,"end":3000,"start":100,"min":12},"active":false,"label":"HSD1","reverseIntensity":false},{"coefficient":1,"family":"linear","color":"FFFFFF","emissionWave":null,"inverted":false,"window":{"max":2094,"end":700,"start":100,"min":13},"active":true,"label":"HSD2","reverseIntensity":false},{"coefficient":1,"family":"linear","color":"00FF00","emissionWave":null,"inverted":false,"window":{"max":2255,"end":1500,"start":200,"min":136},"active":false,"label":"HSD1","reverseIntensity":false},{"coefficient":1,"family":"linear","color":"0000FF","emissionWave":null,"inverted":false,"window":{"max":4095,"end":2000,"start":100,"min":98},"active":false,"label":"HSD2","reverseIntensity":false}],"deltaT":[],"z_start":13,"selected":true,"width":181.